# Load Data from Database Using SQL

Many organizations still use relational database management systems (RDBMS) to store data. A relational database is an application that has a collection of tables, and each table has a row-column structure (similar to a spreadsheet). Databases use Structured Query Language (SQL) to do CRUD operations ("Create", "Read", "Update", or "Delete") on the information in a table (called "querying"). Each table in the database has a name, and the columns within a table are called fields. In this lesson, we will learn how to extract data by passing SQL statements to the database to return the information requested.

In [2]:
#library to use dataframes
import pandas as pd

#library to connect & interact with databases
from sqlalchemy import create_engine, inspect

### Connect to Database

An engine connects to the database and allows queries to be passed in and for information to be read from it. Each database application has its own engine statement; other examples can be found in the [SQL Alchemy documentation](https://docs.sqlalchemy.org/en/13/core/engines.html).

In [3]:
#set the database file location to a variable
db_file = r'datasets/database.sqlite'

# set the connection to SQLite database in a variable
engine = create_engine(f"sqlite:///{db_file}")

In [4]:
#get a list of all the tables in the database
engine.table_names()

['BoardGames',
 'bgg.ldaOut.top.documents',
 'bgg.ldaOut.top.terms',
 'bgg.ldaOut.topics',
 'bgg.topics']

In [5]:
#check if a table exists
engine.has_table('students')

False

In [6]:
#this is used to look at the schema of elements in a database
inspector = inspect(engine)

In [7]:
# get the fields (columns) and their attributes for the table called "test"
#this is a list where each item is a field(column)
print(inspector.get_columns('BoardGames'))

[{'name': 'row_names', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'game.id', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'game.type', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'details.description', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'details.image', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'details.maxplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'details.maxplaytime', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'details.minage', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'details.

In [8]:
#set the table column information to a variable
fields = inspector.get_columns('BoardGames')

In [9]:
#put the information into a dataframe for readability
field_names = pd.DataFrame.from_dict(fields)
field_names

,autoincrement,default,name,nullable,primary_key,type
0,auto,None,row_names,True,0,TEXT
1,auto,None,game.id,True,0,TEXT
2,auto,None,game.type,True,0,TEXT
3,auto,None,details.description,True,0,TEXT
4,auto,None,details.image,True,0,TEXT
5,auto,None,details.maxplayers,True,0,INTEGER
6,auto,None,details.maxplaytime,True,0,INTEGER
7,auto,None,details.minage,True,0,INTEGER
8,auto,None,details.minplayers,True,0,INTEGER
9,auto,None,details.minplaytime,True,0,INTEGER


### Collect Data

In many situations, the easiest method of data collection from a database is to get all the rows from a table, then filter the data in Python. However, in some cases (especially when there are millions of rows) it is better to query the database with a specific condition (for example, to get all the rows in a table where the customers live in the state of Maryland) to reduce the load when the data is read in-memory to Jupyter Notebooks. The `pd.read_sql()` function will send a SQL query statement to the database, then return the information collected and put it into a pandas dataframe.

Different database applications may have their own "dialect" of SQL but the general structure is consistent across all styles. Use this **[cheat sheet](http://www.sqltutorial.org/sql-cheat-sheet/)** to learn more SQL statement commands.

In [10]:
#get 5 rows from the BoardGames table in the database
sql = "SELECT * FROM BoardGames limit 5;"

#the read_sql function takes in the SQL statement for the information requested and the engine (to connect to database)
BG_data_df = pd.read_sql(sql, engine)
BG_data_df

,row_names,game.id,game.type,details.description,details.image,details.maxplayers,details.maxplaytime,details.minage,details.minplayers,details.minplaytime,...,stats.family.arcade.bayesaverage,stats.family.arcade.pos,stats.family.atarist.bayesaverage,stats.family.atarist.pos,stats.family.commodore64.bayesaverage,stats.family.commodore64.pos,stats.subtype.rpgitem.bayesaverage,stats.subtype.rpgitem.pos,stats.subtype.videogame.bayesaverage,stats.subtype.videogame.pos
0,1,1,boardgame,Die Macher is a game about seven sequential po...,//cf.geekdo-images.com/images/pic159509.jpg,5,240,14,3,240,...,None,None,None,None,None,None,None,None,None,None
1,2,2,boardgame,Dragonmaster is a trick-taking card game based...,//cf.geekdo-images.com/images/pic184174.jpg,4,30,12,3,30,...,None,None,None,None,None,None,None,None,None,None
2,3,3,boardgame,"Part of the Knizia tile-laying trilogy, Samura...",//cf.geekdo-images.com/images/pic3211873.jpg,4,60,10,2,30,...,None,None,None,None,None,None,None,None,None,None
3,4,4,boardgame,When you see the triangular box and the luxuri...,//cf.geekdo-images.com/images/pic285299.jpg,4,60,12,2,60,...,None,None,None,None,None,None,None,None,None,None
4,5,5,boardgame,"In Acquire, each player strategically invests ...",//cf.geekdo-images.com/images/pic342163.jpg,6,90,12,3,90,...,None,None,None,None,None,None,None,None,None,None


In [11]:
#query with conditional. Count # of rows in the BoardGames table
sql = 'SELECT COUNT(*) AS game FROM BoardGames'

boardgames_df = pd.read_sql(sql, engine)
boardgames_df

,game
0,90400


In [12]:
#query with conditional. Count # of rows in the bgg.ldaOut.top.documents table
sql = 'SELECT COUNT(*) AS document FROM [bgg.ldaOut.top.documents]'

document_df = pd.read_sql(sql, engine)
document_df

,document
0,288


In [13]:
#query with conditional. Count # of rows in the bgg.ldaOut.top.terms table

sql = 'SELECT COUNT(*) AS term FROM [bgg.ldaOut.top.terms]'

term_df = pd.read_sql(sql, engine)
term_df

,term
0,250


In [14]:
#query with conditional. Count # of rows in the bgg.ldaOut.topics table

sql = 'SELECT COUNT(*) AS topic FROM  [bgg.ldaOut.topics]'

topic_df = pd.read_sql(sql, engine)
topic_df

,topic
0,29229


In [15]:
#query with conditional, count number of rows in the bgg.topics table

sql = 'SELECT COUNT(*) AS topic1 FROM [bgg.topics]'

topic1_df = pd.read_sql(sql, engine)
topic1_df

,topic1
0,29313
